In [1]:
import numpy as np
import pandas as pd

In [2]:
from keras import layers
from keras import Input
from keras.models import Model
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import regularizers
from keras import backend as K
from keras import optimizers


D:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [13]:
question = np.load('question.npy')
negative = np.load('Negative.npy')
positive = np.load('Positive.npy')

In [14]:
rando = np.arange(len(question))
np.random.shuffle(rando)
question,negative,positive = question[rando],negative[rando],positive[rando]

In [43]:
def triplet_loss(y_true,y_pred):
    
    
    positive_pred = y_pred[:,1]
    negative_pred = y_pred[:,0]
    
    sum_loss = 1 + positive_pred - negative_pred
    
    loss = K.maximum(sum_loss,0.0)
    
    return K.sum(loss)
    

In [44]:
def my_log_softmax(x):
    return K.log(K.softmax(x))

# SIEMASE + TRIPLET

In [45]:
left_input = Input(shape=(20,300), name = 'n')
mid_input = Input(shape=(20,300), name = 'q')
right_input = Input(shape=(20,300), name = 'p')

conv1 = layers.Conv1D(128,5, activation='tanh',kernel_regularizer=regularizers.l2(1e-4),input_shape = (None,20,300))
max_pool1 = layers.MaxPooling1D(3)

conv2 = layers.Conv1D(256, 3, activation='tanh',kernel_regularizer=regularizers.l2(1e-4))
max_pool2 = layers.MaxPooling1D(2)
avg_pool2 = layers.AveragePooling1D(2)




flatten = layers.Flatten()


left_output = conv1(left_input)
left_output = max_pool1(left_output)

left_output = conv2(left_output)
left_output = layers.concatenate([max_pool2(left_output),avg_pool2(left_output)], axis = -1)

left_output = flatten(left_output)


mid_output = conv1(mid_input)
mid_output = max_pool1(mid_output)

mid_output = conv2(mid_output)
mid_output = layers.concatenate([max_pool2(mid_output),avg_pool2(mid_output)], axis = -1)

mid_output = flatten(mid_output)


right_output = conv1(right_input)
right_output = max_pool1(right_output)

right_output = conv2(right_output)
right_output = layers.concatenate([max_pool2(right_output),avg_pool2(right_output)], axis = -1)

right_output = flatten(right_output)



merged_L = layers.concatenate([left_output, mid_output], axis=-1)
merged_R = layers.concatenate([right_output, mid_output], axis=-1)


dens1 = layers.Dense(256,activation='tanh',kernel_regularizer=regularizers.l2(1e-4))

droprout1 = layers.Dropout(0.5)

merged_L = droprout1(dens1(merged_L))
merged_R = droprout1(dens1(merged_R))



dens2 = layers.Dense(256, activation='tanh',kernel_regularizer=regularizers.l2(1e-4))
dens_pred = layers.Dense(1, activation='softmax',kernel_regularizer=regularizers.l2(1e-4))

pred_L = dens_pred(dens2(merged_L))
pred_R = dens_pred(dens2(merged_R))




pred = layers.concatenate([pred_L,pred_R], axis = -1)

model = Model(inputs = [left_input, mid_input, right_input], outputs = pred)


# SAMO SIEMASE


In [18]:
left_input = Input(shape=(20,300), name = 'n')
mid_input = Input(shape=(20,300), name = 'q')
# right_input = Input(shape=(20,300), name = 'p')

conv1 = layers.Conv1D(128,5, activation='tanh',kernel_regularizer=regularizers.l2(1e-4),input_shape = (None,20,300))
max_pool1 = layers.MaxPooling1D(3)

conv2 = layers.Conv1D(256, 3, activation='tanh',kernel_regularizer=regularizers.l2(1e-4))
max_pool2 = layers.MaxPooling1D(2)
avg_pool2 = layers.AveragePooling1D(2)

flatten = layers.Flatten()

left_output = conv1(left_input)
left_output = max_pool1(left_output)

left_output = conv2(left_output)
left_output = layers.concatenate([max_pool2(left_output),avg_pool2(left_output)], axis = -1)

left_output = flatten(left_output)


mid_output = conv1(mid_input)
mid_output = max_pool1(mid_output)

mid_output = conv2(mid_output)
mid_output = layers.concatenate([max_pool2(mid_output),avg_pool2(mid_output)], axis = -1)

mid_output = flatten(mid_output)



merged_L = layers.concatenate([left_output, mid_output], axis=-1)
# merged_R = layers.concatenate([right_output, mid_output], axis=-1)


dens1 = layers.Dense(256,activation='tanh',kernel_regularizer=regularizers.l2(1e-4))

droprout1 = layers.Dropout(0.5)

merged_L = droprout1(dens1(merged_L))
# merged_R = droprout1(dens1(merged_R))



dens2 = layers.Dense(256, activation='tanh',kernel_regularizer=regularizers.l2(1e-4))
dens_pred = layers.Dense(1, activation=my_log_softmax,kernel_regularizer=regularizers.l2(1e-4))

# pred_L = dens_pred(dens2(merged_L))
# pred_R = dens_pred(dens2(merged_R))

dens_pred = dens_pred(dens2(merged_L))


# pred = layers.concatenate([pred_L,pred_R], axis = -1)

model = Model(inputs = [left_input, mid_input], outputs = dens_pred)

In [24]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
n (InputLayer)                  (None, 20, 300)      0                                            
__________________________________________________________________________________________________
q (InputLayer)                  (None, 20, 300)      0                                            
__________________________________________________________________________________________________
p (InputLayer)                  (None, 20, 300)      0                                            
__________________________________________________________________________________________________
conv1d_9 (Conv1D)               (None, 16, 128)      192128      n[0][0]                          
                                                                 q[0][0]                          
          

In [46]:
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer=optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False))

In [47]:
etyk = np.random.randint(2, size=len(question)).astype(float)
etyk[etyk > 0.0] = .99

In [48]:
model.fit({'n' : negative, 'q' : question, 'p' : positive},
          etyk, #cokolwiek, nie istotne przy loss function          
          epochs=10, 
          batch_size = 32, # cokolwiek w sumie
          verbose=1)

Epoch 1/10
8484/8484 [==============================] - 8s 945us/step - loss: 0.6953
Epoch 2/10
8484/8484 [==============================] - 7s 865us/step - loss: 0.6932
Epoch 3/10
8484/8484 [==============================] - 7s 862us/step - loss: 0.6932
Epoch 4/10
8484/8484 [==============================] - 7s 875us/step - loss: 0.6934
Epoch 5/10
8484/8484 [==============================] - 8s 888us/step - loss: 0.6935
Epoch 6/10
8484/8484 [==============================] - 8s 935us/step - loss: 0.6935
Epoch 7/10
8484/8484 [==============================] - 7s 882us/step - loss: 0.6935
Epoch 8/10
8484/8484 [==============================] - 7s 883us/step - loss: 0.6935
Epoch 9/10
8484/8484 [==============================] - 8s 953us/step - loss: 0.6934
Epoch 10/10
8484/8484 [==============================] - 9s 1ms/step - loss: 0.6935


In [41]:
p = model.predict({'n' : negative, 'q' : question})#, 'p' : positive})

In [42]:
p

array([[1., 1.],
       [1., 1.],
       [1., 1.],
       ...,
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32)